In [ ]:
import pandas as pd
import altair as alt

# Carica i dati
df = pd.read_excel(file_path)

# Trasforma i dati da formato wide a long
df_long = df.melt(id_vars='Genere', var_name='Comportamento', value_name='Proporzione')

# Raggruppa per comportamento e somma le proporzioni di uomini e donne
df_grouped = df_long.groupby('Comportamento')['Proporzione'].sum().reset_index()
df_grouped.columns = ['Comportamento', 'Proporzione Totale']

# Calcola la percentuale sul totale complessivo per la visualizzazione
total_sum = df_grouped['Proporzione Totale'].sum()
df_grouped['Percentuale'] = (df_grouped['Proporzione Totale'] / total_sum) * 100

# Crea il grafico a ciambella (Donut Chart)
base = alt.Chart(df_grouped).encode(
    theta=alt.Theta("Proporzione Totale", stack=True)
).properties(
    title='Distribuzione Relativa dei Comportamenti di Phishing (Uomini e Donne Combinati)'
)

pie = base.mark_arc(outerRadius=120, innerRadius=60).encode(
    color=alt.Color("Comportamento", title="Comportamento", scale=alt.Scale(range=['#9932CC', '#FF00FF', '#FFC0CB'])), # Viola scuro (DarkOrchid), Fucsia, Rosa (Pink)
    order=alt.Order("Proporzione Totale", sort="descending"),
    tooltip=[
        alt.Tooltip("Comportamento", title="Comportamento"),
        alt.Tooltip("Proporzione Totale", title="Proporzione (Somma)", format=".2f"),
        alt.Tooltip("Percentuale", title="Percentuale", format=".1f")
    ]
)

text = base.mark_text(radius=140).encode(
    text=alt.Text("Percentuale", format=".1f"),
    order=alt.Order("Proporzione Totale", sort="descending"),
    color=alt.value("black")
)

chart = pie + text

# Salva il grafico
chart.save('distribuzione_comportamenti_phishing.json')

# Visualizza il grafico direttamente nel notebook
chart